In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import csv
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

# Import API key
#from api_keys import geoapify_key


In [2]:
# Specify the path to your CSV file
csvpath = 'Resources/reduced.csv'

# Read the CSV file into a DataFrame
data = pd.read_csv(csvpath)
data_df = pd.DataFrame(data)
# Display the DataFrame
data_df


,Unnamed: 0,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,357091,Texas,Female,Very good,0.0,30.0,Within past year (anytime less than 12 months ...,Yes,6.0,None of them,...,1.65,58.97,21.63,No,No,No,No,"Yes, received tetanus shot but not sure what type",No,Yes
1,199859,Minnesota,Female,Good,6.0,25.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,...,1.55,56.70,23.62,Yes,Yes,No,Yes,"Yes, received tetanus shot but not sure what type",Yes,No
2,347964,Texas,Female,Good,10.0,20.0,Within past year (anytime less than 12 months ...,Yes,12.0,"6 or more, but not all",...,1.60,61.23,23.91,Yes,No,No,Yes,"Yes, received Tdap",No,No
3,407343,Washington,Female,Fair,0.0,19.0,Within past year (anytime less than 12 months ...,No,8.0,None of them,...,1.57,NaN,NaN,Yes,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,Tested positive using home test without a heal...
4,211281,Mississippi,Male,Very good,4.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,...,1.85,95.25,27.71,Yes,No,Yes,No,"Yes, received tetanus shot, but not Tdap",No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111278,179391,Massachusetts,Male,Excellent,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,...,1.70,74.84,25.84,Yes,Yes,No,No,"Yes, received tetanus shot but not sure what type",No,Tested positive using home test without a heal...
111279,36107,California,Female,Excellent,0.0,0.0,5 or more years ago,Yes,8.0,None of them,...,1.75,65.77,21.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111280,407783,Washington,Female,Good,0.0,10.0,Within past year (anytime less than 12 months ...,Yes,6.0,None of them,...,1.65,68.04,24.96,No,Yes,No,No,"Yes, received Tdap",No,No
111281,417600,West Virginia,Female,Good,0.0,10.0,Within past year (anytime less than 12 months ...,Yes,8.0,None of them,...,1.55,52.16,21.73,Yes,No,Yes,No,"No, did not receive any tetanus shot in the pa...",No,No


In [3]:
print(data_df.columns)

Index(['Unnamed: 0', 'State', 'Sex', 'GeneralHealth', 'PhysicalHealthDays',
       'MentalHealthDays', 'LastCheckupTime', 'PhysicalActivities',
       'SleepHours', 'RemovedTeeth', 'HadHeartAttack', 'HadAngina',
       'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'HeightInMeters', 'WeightInKilograms', 'BMI', 'AlcoholDrinkers',
       'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap',
       'HighRiskLastYear', 'CovidPos'],
      dtype='object')


In [4]:
#Remove unneeded columns
cleaned_data_df=data_df[['State', 'Sex','GeneralHealth', 'PhysicalHealthDays', 'MentalHealthDays',	'PhysicalActivities', 
                         'SleepHours','HadDepressiveDisorder','BMI', 'AlcoholDrinkers', 'ECigaretteUsage','SmokerStatus']]
cleaned_data_df

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,HadDepressiveDisorder,BMI,AlcoholDrinkers,ECigaretteUsage,SmokerStatus
0,Texas,Female,Very good,0.0,30.0,Yes,6.0,No,21.63,No,NaN,Never smoked
1,Minnesota,Female,Good,6.0,25.0,Yes,7.0,Yes,23.62,Yes,Use them every day,Current smoker - now smokes some days
2,Texas,Female,Good,10.0,20.0,Yes,12.0,No,23.91,Yes,Never used e-cigarettes in my entire life,Never smoked
3,Washington,Female,Fair,0.0,19.0,No,8.0,No,NaN,Yes,Never used e-cigarettes in my entire life,Former smoker
4,Mississippi,Male,Very good,4.0,0.0,Yes,7.0,No,27.71,Yes,Never used e-cigarettes in my entire life,Never smoked
...,...,...,...,...,...,...,...,...,...,...,...,...
111278,Massachusetts,Male,Excellent,0.0,0.0,Yes,7.0,No,25.84,Yes,Not at all (right now),Never smoked
111279,California,Female,Excellent,0.0,0.0,Yes,8.0,No,21.41,NaN,Not at all (right now),Current smoker - now smokes some days
111280,Washington,Female,Good,0.0,10.0,Yes,6.0,No,24.96,No,Never used e-cigarettes in my entire life,Never smoked
111281,West Virginia,Female,Good,0.0,10.0,Yes,8.0,No,21.73,Yes,Never used e-cigarettes in my entire life,Former smoker


In [5]:
#Check for columns with 'Nan' values
non_nan_counts = cleaned_data_df.count()
print(non_nan_counts)

State                    111283
Sex                      111283
GeneralHealth            111004
PhysicalHealthDays       108526
MentalHealthDays         109110
PhysicalActivities       111021
SleepHours               109954
HadDepressiveDisorder    110565
BMI                       99064
AlcoholDrinkers           99770
ECigaretteUsage          102452
SmokerStatus             102501
dtype: int64


In [6]:
#Remove 'Nan' values from data
cleaned_data2_df=cleaned_data_df.dropna()
cleaned_data2_df

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,HadDepressiveDisorder,BMI,AlcoholDrinkers,ECigaretteUsage,SmokerStatus
1,Minnesota,Female,Good,6.0,25.0,Yes,7.0,Yes,23.62,Yes,Use them every day,Current smoker - now smokes some days
2,Texas,Female,Good,10.0,20.0,Yes,12.0,No,23.91,Yes,Never used e-cigarettes in my entire life,Never smoked
4,Mississippi,Male,Very good,4.0,0.0,Yes,7.0,No,27.71,Yes,Never used e-cigarettes in my entire life,Never smoked
5,California,Female,Very good,0.0,15.0,Yes,7.0,No,24.69,Yes,Not at all (right now),Never smoked
6,Georgia,Female,Good,0.0,6.0,Yes,6.0,No,31.41,No,Never used e-cigarettes in my entire life,Never smoked
...,...,...,...,...,...,...,...,...,...,...,...,...
111276,Florida,Female,Good,0.0,30.0,Yes,6.0,No,28.29,No,Never used e-cigarettes in my entire life,Never smoked
111278,Massachusetts,Male,Excellent,0.0,0.0,Yes,7.0,No,25.84,Yes,Not at all (right now),Never smoked
111280,Washington,Female,Good,0.0,10.0,Yes,6.0,No,24.96,No,Never used e-cigarettes in my entire life,Never smoked
111281,West Virginia,Female,Good,0.0,10.0,Yes,8.0,No,21.73,Yes,Never used e-cigarettes in my entire life,Former smoker


In [7]:
#Make sure 'Nan' values are removed
non_nan_counts = cleaned_data2_df.count()
print(non_nan_counts)

State                    87906
Sex                      87906
GeneralHealth            87906
PhysicalHealthDays       87906
MentalHealthDays         87906
PhysicalActivities       87906
SleepHours               87906
HadDepressiveDisorder    87906
BMI                      87906
AlcoholDrinkers          87906
ECigaretteUsage          87906
SmokerStatus             87906
dtype: int64


In [9]:
states=cleaned_data2_df['State'].unique()
print(states)

['Minnesota' 'Texas' 'Mississippi' 'California' 'Georgia' 'Wisconsin'
 'Iowa' 'Colorado' 'Arizona' 'Michigan' 'New Hampshire' 'Indiana'
 'District of Columbia' 'Nebraska' 'Washington' 'Missouri' 'Massachusetts'
 'Puerto Rico' 'Idaho' 'Alabama' 'Tennessee' 'Montana' 'Connecticut'
 'South Dakota' 'Utah' 'Oklahoma' 'Florida' 'South Carolina' 'Virginia'
 'Ohio' 'North Dakota' 'Kansas' 'Maryland' 'Oregon' 'Vermont' 'Maine'
 'Kentucky' 'Nevada' 'Rhode Island' 'New York' 'West Virginia' 'Delaware'
 'North Carolina' 'Alaska' 'Hawaii' 'New Jersey' 'Pennsylvania'
 'Louisiana' 'New Mexico' 'Illinois' 'Arkansas' 'Wyoming' 'Virgin Islands'
 'Guam']


In [10]:
#Remove Guam, Puerto Rico and VI
values_to_remove=['Guam','Puerto Rico','Virgin Islands','District of Columbia']
cleaned_data3_df=cleaned_data2_df[~cleaned_data2_df['State'].isin(values_to_remove)]
cleaned_data3_df

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,HadDepressiveDisorder,BMI,AlcoholDrinkers,ECigaretteUsage,SmokerStatus
1,Minnesota,Female,Good,6.0,25.0,Yes,7.0,Yes,23.62,Yes,Use them every day,Current smoker - now smokes some days
2,Texas,Female,Good,10.0,20.0,Yes,12.0,No,23.91,Yes,Never used e-cigarettes in my entire life,Never smoked
4,Mississippi,Male,Very good,4.0,0.0,Yes,7.0,No,27.71,Yes,Never used e-cigarettes in my entire life,Never smoked
5,California,Female,Very good,0.0,15.0,Yes,7.0,No,24.69,Yes,Not at all (right now),Never smoked
6,Georgia,Female,Good,0.0,6.0,Yes,6.0,No,31.41,No,Never used e-cigarettes in my entire life,Never smoked
...,...,...,...,...,...,...,...,...,...,...,...,...
111276,Florida,Female,Good,0.0,30.0,Yes,6.0,No,28.29,No,Never used e-cigarettes in my entire life,Never smoked
111278,Massachusetts,Male,Excellent,0.0,0.0,Yes,7.0,No,25.84,Yes,Not at all (right now),Never smoked
111280,Washington,Female,Good,0.0,10.0,Yes,6.0,No,24.96,No,Never used e-cigarettes in my entire life,Never smoked
111281,West Virginia,Female,Good,0.0,10.0,Yes,8.0,No,21.73,Yes,Never used e-cigarettes in my entire life,Former smoker


In [11]:
#Verify the rows were removed
Number_of_states=cleaned_data3_df['State'].nunique()
print(Number_of_states)

50


In [12]:
#Randomly sample a portion of the data to make it easier to work with (optional); frac value is a percentage 
Random_df=cleaned_data3_df.sample(frac=0.1, random_state=42)
Random_df

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,HadDepressiveDisorder,BMI,AlcoholDrinkers,ECigaretteUsage,SmokerStatus
11160,Alaska,Female,Poor,30.0,10.0,Yes,6.0,No,25.34,No,Not at all (right now),Current smoker - now smokes every day
39460,Florida,Male,Excellent,2.0,0.0,Yes,6.0,No,26.58,No,Never used e-cigarettes in my entire life,Never smoked
16355,New York,Male,Good,0.0,0.0,Yes,8.0,No,30.52,Yes,Never used e-cigarettes in my entire life,Never smoked
48492,Connecticut,Male,Good,0.0,0.0,Yes,7.0,No,29.84,Yes,Not at all (right now),Former smoker
92216,Texas,Male,Very good,2.0,0.0,Yes,10.0,No,22.15,No,Never used e-cigarettes in my entire life,Never smoked
...,...,...,...,...,...,...,...,...,...,...,...,...
81740,Minnesota,Female,Excellent,0.0,0.0,Yes,7.0,No,20.05,Yes,Never used e-cigarettes in my entire life,Never smoked
1758,Hawaii,Female,Very good,0.0,1.0,Yes,6.0,Yes,30.90,Yes,Never used e-cigarettes in my entire life,Never smoked
84379,Maine,Female,Good,0.0,2.0,Yes,6.0,Yes,18.60,No,Never used e-cigarettes in my entire life,Current smoker - now smokes every day
4059,Washington,Female,Very good,3.0,0.0,Yes,5.0,No,47.47,No,Never used e-cigarettes in my entire life,Never smoked
